In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [32]:
dataset  = pd.read_csv("Social_Network_Ads.csv")

In [33]:
dataset = dataset.drop(columns="User ID")

In [34]:
independent = dataset[['Age', 'EstimatedSalary', 'Gender']]
dependent = dataset[['Purchased']].values.ravel()

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent,dependent,test_size=0.3)

In [36]:
from sklearn.svm import  SVC

In [37]:
preprocessor = ColumnTransformer([
('one_hot',OneHotEncoder(drop='first'),['Gender']),
 ('sc',StandardScaler(),['Age', 'EstimatedSalary'])
]
)

In [38]:
pipeline = Pipeline([
('preprocess',preprocessor),
('svc',SVC(random_state=42))
]
)

In [39]:
param_grid = {
    'svc__C': [0.1, 1, 5, 10, 100],
    'svc__kernel': ['linear', 'rbf', 'poly'],
    'svc__gamma': ['scale', 'auto'],
    
    'svc__class_weight': [None, 'balanced']
}

In [40]:
model = GridSearchCV(estimator=pipeline,param_grid=param_grid,n_jobs=-1,refit=True,verbose=3)

In [41]:
model.fit(X_train,y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


GridSearchCV(estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('one_hot',
                                                                         OneHotEncoder(drop='first'),
                                                                         ['Gender']),
                                                                        ('sc',
                                                                         StandardScaler(),
                                                                         ['Age',
                                                                          'EstimatedSalary'])])),
                                       ('svc', SVC(random_state=42))]),
             n_jobs=-1,
             param_grid={'svc__C': [0.1, 1, 5, 10, 100],
                         'svc__class_weight': [None, 'balanced'],
                         'svc__gamma': ['scale', 'auto'],
                         'svc__kernel': ['linear', 'rbf', 'poly']},
             verbose=3)

In [42]:
y_pred = model.predict(X_test)

In [43]:
from sklearn.metrics import confusion_matrix,classification_report

In [44]:
confusion_matrix(y_test,y_pred)


array([[67,  7],
       [ 2, 44]], dtype=int64)

In [45]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.97      0.91      0.94        74
           1       0.86      0.96      0.91        46

    accuracy                           0.93       120
   macro avg       0.92      0.93      0.92       120
weighted avg       0.93      0.93      0.93       120



In [63]:
report = classification_report(y_test,y_pred,output_dict=True)

In [65]:
import json

In [69]:
with(open("SVC_report.json","w")) as f:
    json.dump(report,f)